This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
response = requests.get(f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-01-02&api_key={API_KEY}')
print(response)
type(response)

<Response [200]>


requests.models.Response

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
my_dict = response.json()
print(my_dict, end='\n\n')
type(my_dict)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2017-01-01', 'end_date': '2017-01-02', 'frequency': 'daily', 'data': [['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None}}



dict

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#Task 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, 
#for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

#Data for the 2017 year is imported using the requests library and saved in response
response = requests.get(f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}')
type(response)

requests.models.Response

In [6]:
#Task 2: Convert the returned JSON object into a Python dictionary.

#Then to create a Python dictionary, we use the .json() method from the Requests package, on the response.
my_dict = response.json()
type(my_dict)

dict

In [7]:
#Task 3: Calculate what the highest and lowest opening prices were for the stock in this period.

#First, let's get the dictionary inside the key 'dataset_data'
my_nested_dict = my_dict['dataset_data']
# Now, we take a list of columns and a list of data to create a dictionary where values are lists for every key
data_columns = my_nested_dict['column_names']
data = my_nested_dict['data']
dict_of_lists = {data_columns[i] : [data[j][i] for j in range(len(data))]for i in range(len(data_columns))}
#After creating the dictionary with column labels as keys and lists of values as values, 
#we extract the list of values from the column 'Open'
open_list = dict_of_lists['Open']
#Then we just get the max and min values for the list, ignoring the None values
print(max(i for i in open_list if i is not None))
print(min(i for i in open_list if i is not None))

53.11
34.0


In [8]:
#Task 4: What was the largest change in any one day (based on High and Low price)?

#First we get the high price and low prices lists from the dict_of_lists dictionary
high_list = dict_of_lists['High']
low_list = dict_of_lists['Low']
#Now that we have both lists we just create a new list by inserting the substraction of every high  low values
price_diff_list = [high - low for high, low in zip(high_list, low_list)]
#Finally just print the max value from the differences list
print(max(price_diff_list))

2.8100000000000023


In [9]:
#Task 5: What was the largest change between any two days (based on Closing Price)?

#First let's make a list of closing prices
close_list = dict_of_lists['Close']
#Then we make a new list containing the differences, in absolute value, between one day and the next
#for the whole period
close_diff_list = [round(abs(close_list[i-1] - close_list[i]),2) for i in range(1, len(close_list))]
#Now we print the max value from the new list to get the largest change in closing price
print(max(close_diff_list))

2.56


In [10]:
#Task 6: What was the average daily trading volume during this year?

#In this case, we define a function to calculate the average daily trading volume
def average(array, length):
    avg = sum(array)/length
    return avg

#Once we have the function to calculate the average, just input the list of values and the length of the list
#to calculate the average daily trading volume
traded_vol_list = dict_of_lists['Traded Volume']
length = len(traded_vol_list)
print(average(traded_vol_list, length))

89124.33725490196


In [11]:
#Task 7: (Optional) What was the median trading volume during this year. 
#(Note: you may need to implement your own function for calculating the median.)

#In this case, we define a function for calculating the median
def median(array, length):
    #First we put the array in ascending order
    ordered_array = sorted(array)
    #We check if the length of the array is even or odd and designate the middle point of the array
    if length % 2 == 1:
        middle = int((length + 1)/2)
    else:
        middle = int(length/2)
    #Return the value in the middle of the array   
    return ordered_array[middle]
#Print the value to the screen by using the median function with the volume list and length of this list as parameters    
print(median(traded_vol_list, len(traded_vol_list)))

76600.0
